# **Import libraries**

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import math
import cv2

import argparse
import random
import gc
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

from tqdm import tqdm
pd.options.display.max_colwidth = 1000
tqdm.pandas()

import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Add, Concatenate, Input
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D, LSTM, Reshape
from keras.layers import BatchNormalization, SeparableConv2D, DepthwiseConv2D, LeakyReLU, GlobalAveragePooling2D
from keras import optimizers
from keras import backend as K
from keras import layers

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.regularizers import L2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Make sure we are able to handle large datasets
import resource

low, high = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (high, high))

# Initialize constants and lists

In [ ]:
activity_map = {'c0': 'Safe driving',
                'c1': 'Texting - right',
                'c2': 'Talking on the phone - right',
                'c3': 'Texting - left',
                'c4': 'Talking on the phone - left',
                'c5': 'Operating the radio',
                'c6': 'Drinking',
                'c7': 'Reaching behind',
                'c8': 'Hair and makeup',
                'c9': 'Talking to passenger'}
class_mapping = {'c0': 0,
                'c1': 1,
                'c2': 2,
                'c3': 3,
                'c4': 4,
                'c5': 5,
                'c6': 6,
                'c7': 7,
                'c8': 8,
                'c9': 9}

# Algorithm hyperparameters
num_epochs = 70
batch_size = 64
width = 256
temperature = 0.1

# Stronger augmentations for contrastive, weaker ones for supervised training
contrastive_augmentation = {
    "min_area": 0.75, 
    "brightness": 0.5, 
    "jitter": 0.2
}

classification_augmentation = {
    "min_area": 0.8,
    "brightness": 0.3,
    "jitter": 0.1,
}

IMG_DIM = 208
CHANNEL_SIZE = 3
BATCH_SIZE = 64

**AUC data test**

In [ ]:
auc_data_path = ''/kaggle/input/auc-dataset/v2_cam1_cam2_split_by_driver/Camera_1'

driver_imgs_list_train_AUC = pd.read_csv(os.path.join(auc_data_path, 'train_imgs_list_cam1.csv'))

In [ ]:
list_train_img_AUC = glob.glob(os.path.join(auc_data_path, "train", "*",  "*.jpg"))
print("Total number of Train Images is -------->", len(list_train_img_AUC))

In [ ]:
driver_imgs_list_train_AUC['ImgPath'] = driver_imgs_list_train_AUC['img'].progress_apply(lambda x: [i for i in  list_train_img_AUC if x in i][0])

driver_imgs_list_train_AUC['class'] = driver_imgs_list_train_AUC['classname'].replace(activity_map)
df_AUC = driver_imgs_list_train_AUC.copy()

#del driver_imgs_list_train, list_train_img

In [ ]:
df_AUC.head(100)

In [ ]:
# BatPlot and Piechart
class_freq_count = df_AUC['class'].value_counts()

class_freq_count.plot(kind='bar', label='index')
plt.title('Sample Per Class')
plt.show()

class_freq_count.plot(kind='pie', autopct='%1.1f%%', label='index')
plt.title('Sample % per class')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
train_dataset_AUC, test_valid_dataset_AUC = train_test_split(df_AUC ,test_size = 0.5 , random_state = 42, shuffle = True,
                                              stratify = df_AUC['class'])
valid_dataset_AUC, test_dataset_AUC = train_test_split(test_valid_dataset_AUC, test_size = 0.5, random_state = 42, shuffle = True,
                                              stratify = test_valid_dataset_AUC['class'])

In [ ]:
len(train_dataset_AUC), len(valid_dataset_AUC), len(test_dataset_AUC)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
'''
training_set = train_datagen.flow_from_dataframe(
                        dataframe = train_dataset_AUC,
                        x_col = 'ImgPath',
                        y_col = 'class',
                        target_size=(IMG_DIM,IMG_DIM),
                        class_mode = 'categorical',
                        batch_size= 64)

validation_set = valid_datagen.flow_from_dataframe(
                dataframe = valid_dataset_AUC,
                x_col = 'ImgPath',
                y_col = 'class',
                target_size = (IMG_DIM, IMG_DIM),
                class_mode = 'categorical',
                batch_size = 64)
'''
test_set_AUC = test_datagen.flow_from_dataframe(
                dataframe = test_dataset_AUC,
                x_col = "ImgPath",
                y_col = 'class',
                target_size = (IMG_DIM, IMG_DIM),
                class_mode = 'categorical',
                batch_size = 64)

In [ ]:
test_dataset_AUC = test_dataset_AUC['ImgPath class'.split()].copy().reset_index(drop=True)
print("shape_of_the_valid_dataset", test_dataset_AUC.shape)
test_dataset_AUC.sample(1)

In [ ]:
test_dataset_AUC['prediction'] = test_dataset_AUC['ImgPath'].progress_apply(lambda x:model_prediction(x, finetuning_model))
test_dataset_AUC.head(5)

In [ ]:
y_true_test = list(test_dataset_AUC['class'])
y_pred_test = list(test_dataset_AUC['prediction'])

In [ ]:
c_m_test_AUC = confusion_matrix(y_true_test, y_pred_test)

In [ ]:
# Magic function that renders the figure in a jupyter notebook
# instead of displaying a figure object
%matplotlib inline


# Setting default size of the plot
# Setting default fontsize used in the plot
plt.rcParams['figure.figsize'] = (10.0, 9.0)
plt.rcParams['font.size'] = 20


# Implementing visualization of Confusion Matrix
display_c_m = ConfusionMatrixDisplay(c_m_test_AUC, display_labels=sorted_labels)


# Plotting Confusion Matrix
# Setting colour map to be used
display_c_m.plot(cmap='OrRd', xticks_rotation=25)
# Other possible options for colour map are:
# 'autumn_r', 'Blues', 'cool', 'Greens', 'Greys', 'PuRd', 'copper_r'

# Setting fontsize for xticks and yticks
plt.xticks(np.arange(len(label_map)), label_map.keys(), fontsize=10, rotation=25)
plt.yticks(np.arange(len(label_map)), label_map.keys(), fontsize=10)


# Giving name to the plot
plt.title('Confusion Matrix on Test data from SF3D', fontsize=24)

# Showing the plot
plt.show()

In [ ]:
# !pip install pandas tabulate

from tabulate import tabulate
from sklearn.metrics import classification_report

report = classification_report(y_true_test, y_pred_test, output_dict=True)

# Chuyển từ điển vào DataFrame
df = pd.DataFrame(report).transpose()
df_filtered = df.drop('accuracy')

# In kết quả dưới dạng bảng sử dụng tabulate
table = tabulate(df_filtered, headers='keys', tablefmt='fancy_grid')
print(table)